In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

In [14]:
def calcular_matriz_covarianza_escalar(retornos):
    """
    Calcula una matriz de covarianza escalar para un conjunto dado de retornos de activos.
    
    Parameters:
    retornos (pd.DataFrame): DataFrame de pandas que contiene los retornos de los activos.
                             Cada columna representa un activo y cada fila representa una observación.
    
    Returns:
    np.ndarray: Una matriz de covarianza escalar donde todas las varianzas son iguales a la varianza
                media de los activos y todas las covarianzas son cero.
    
    Raises:
    ValueError: Si 'retornos' no es un DataFrame.
    ValueError: Si 'retornos' tiene menos de una columna (activo).
    ValueError: Si 'retornos' contiene alguna fila con todos valores NaN.
    """
    
    # Verificar que 'retornos' sea un DataFrame
    if not isinstance(retornos, pd.DataFrame):
        raise ValueError("El parámetro 'retornos' debe ser un pd.DataFrame.")
    
    # Verificar que hay suficientes activos
    if retornos.shape[1] < 1:
        raise ValueError("El DataFrame 'retornos' debe contener los retornos de al menos un activo.")
    
    # Verificar que no hay filas completamente vacías
    if retornos.dropna(how='all').empty:
        raise ValueError("El DataFrame 'retornos' no debe tener filas completamente vacías.")

    # Calcular la varianza media de todos los activos
    varianza_media = retornos.var().mean()
    
    # Obtener el número de activos
    n_activos = retornos.shape[1]
    
    # Crear una matriz de covarianza escalar
    matriz_covarianza_escalar = np.zeros((n_activos, n_activos))
    np.fill_diagonal(matriz_covarianza_escalar, varianza_media)
    
    return matriz_covarianza_escalar

In [18]:
def calcular_shrinkage_ledoit_wolf(retornos):
    """
    Calcula la matriz de covarianza shrinked utilizando el método de Ledoit-Wolf.
    
    Parameters:
    retornos (pd.DataFrame): DataFrame que contiene los retornos de los activos,
                             donde cada columna representa un activo y cada fila
                             representa una observación.
                             
    Returns:
    np.ndarray: Matriz de covarianza shrinked, donde la varianza de cada activo
                está ajustada para mejorar la estabilidad y precisión de la matriz.
    
    Raises:
    ValueError: Si 'retornos' no es un DataFrame.
    ValueError: Si 'retornos' tiene menos de dos columnas o menos de una fila.
    """
    
    # Verificaciones de integridad de la entrada
    if not isinstance(retornos, pd.DataFrame):
        raise ValueError("El parámetro 'retornos' debe ser un pd.DataFrame.")
    if retornos.shape[1] < 2 or retornos.shape[0] < 1:
        raise ValueError("El DataFrame 'retornos' debe contener al menos dos activos y una observación.")

    # Calcula la matriz de covarianza muestral
    matriz_covarianza_muestral = retornos.cov().values

    # Estimar la matriz de covarianza escalar
    matriz_covarianza_escalar = calcular_matriz_covarianza_escalar(retornos)
    
    # Número de observaciones y activos
    n, p = retornos.shape
    
    # Media de cada activo
    retornos_media = retornos.mean()
    
    # Centrar los datos
    retornos_cent = retornos - retornos_media
    
    # Calcular la matriz de varianza de los retornos centrados usando covarianza con ddof=0
    matriz_varianza = retornos_cent.cov(ddof=0).values
    
    # Diferencia entre la matriz de varianza y la matriz escalar
    matriz_diferencia = matriz_varianza - matriz_covarianza_escalar
    
    # Calcular el numerador (suma de cuadrados de las diferencias)
    numerador = np.sum(matriz_diferencia ** 2)
    
    # Calcular el denominador (error de predicción de la matriz muestral)
    denominador = 0
    for i in range(n):
        x = retornos_cent.iloc[i].values
        matriz_omega = np.outer(x, x)
        matriz_omega -= matriz_varianza
        denominador += np.sum(matriz_omega ** 2)
    denominador /= n
    
    # Calcular el parámetro de encogimiento
    if denominador != 0:
        shrinkage = numerador / denominador
    else:
        shrinkage = 0
    shrinkage = max(0, min(1, shrinkage))  # Asegura que el encogimiento esté entre 0 y 1
    
    # Calcular la matriz de covarianza shrinked
    matriz_covarianza_shrinked = shrinkage * matriz_covarianza_escalar + (1 - shrinkage) * matriz_covarianza_muestral
    
    return matriz_covarianza_shrinked


In [ ]:
def cartera_min_vol (ret, metodo=):
    
    ''' Función que calcula la cartera de mínima varianza para un DataFrame de rendimientos
    ret: DataFrame de rendimientos
    Retorna pesos_ajustados: Array con los pesos de la cartera de mínima varianza'''
    
    if isinstance(ret, pd.DataFrame): # Verifico que el argumento sea un DataFrame
    
        num_act = ret.shape[1]
        matriz_cov = ret.cov().to_numpy()
        
            #Variables de decisión
        pesos = cp.Variable(num_act)
        
        #Restricciones
        constraints = [pesos >= 0,
                    cp.sum(pesos) == 1,
                    ]
        
        riesgo = cp.quad_form(pesos, matriz_cov) # Riesgo de la cartera
        objective = cp.Minimize(riesgo) # Minimizar la varianza

        #Problema y resuelvo
        prob = cp.Problem(objective, constraints)
        resultado = prob.solve()

        pesos_ajustados = np.array([np.round(x, 3) if x > 10**-4 else 0  for x in pesos.value]) #Pongo a cero los pesos menores a 10^-4 y redondeo a 3 decimales

        return pesos_ajustados
    
    else:
        raise ValueError('La función cartera_min_vol solo acepta un DataFrame como argumento') # Si el argumento no es un DataFrame, lanzo un error

In [21]:
retornos_media = retornos_activos_mes_ini.mean()
retornos_cent = retornos_activos_mes_ini - retornos_media

x = retornos_cent.iloc[0].values
matriz_omega = np.outer(x, x)

In [24]:
matriz_omega.shape, retornos_activos_mes_ini.shape

((50, 50), (351, 50))

In [2]:
start_date = '1995-01-01'
precios_indice = yf.download("SPY", start=start_date)[["Adj Close"]] # Precios ajustados al cierre

[*********************100%%**********************]  1 of 1 completed


In [3]:
filepath = 'https://raw.githubusercontent.com/alfonso-santos/microcredencial-carteras-python-2023/main/Tema_5_APT/data/sp500_tickers.csv'
tickers_sp500 = list(pd.read_csv(filepath))

precios = yf.download(tickers_sp500, start=start_date)['Adj Close']

precios_activos_sp500 = precios.copy()
precios_activos_sp500.dropna(axis=1, inplace=True)
#ret_activos_sp500 = np.log(precios_activos_sp500).diff().dropna()

[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['CDAY', 'BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1995-01-01 -> 2024-04-28)')


In [4]:
num_act_max = 100

num_columnas = precios_activos_sp500.shape[1]

# Generar índices aleatorios para seleccionar num_act_max activos sin repetición

# Fijar la semilla del generador de números aleatorios
np.random.seed(42)  # Puedes usar cualquier número entero como semilla

indices_aleatorios = np.random.choice(num_columnas, size=50, replace=False)

# Seleccionar las columnas del array original usando los índices aleatorios
precios_activos_select = precios_activos_sp500.iloc[:, indices_aleatorios]


In [9]:
fechas_primer_dia = precios_activos_select.groupby([precios_activos_select.index.year, precios_activos_select.index.month]).apply(lambda x: x.index.min()).values
precios_activos_mes_ini = precios_activos_select.loc[fechas_primer_dia]
retornos_activos_mes_ini = np.log(precios_activos_mes_ini).diff().dropna()

In [19]:
calcular_shrinkage_ledoit_wolf(retornos_activos_mes_ini)

array([[0.00425563, 0.00182844, 0.00180102, ..., 0.00183856, 0.00113357,
        0.00247931],
       [0.00182844, 0.00701708, 0.00210287, ..., 0.00216841, 0.00181315,
        0.00308253],
       [0.00180102, 0.00210287, 0.01563449, ..., 0.00287252, 0.00079934,
        0.00305878],
       ...,
       [0.00183856, 0.00216841, 0.00287252, ..., 0.0051405 , 0.00166928,
        0.00259277],
       [0.00113357, 0.00181315, 0.00079934, ..., 0.00166928, 0.00642201,
        0.0018885 ],
       [0.00247931, 0.00308253, 0.00305878, ..., 0.00259277, 0.0018885 ,
        0.00835626]])

In [12]:

np.issubdtype(retornos_activos_mes_ini.dtypes, np.number)

False